# Deploying Tensorflow models on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application.) See more [here](https://docs.verta.ai/verta/registry/concepts).

This notebook provides an example of how to deploy a Tensorflow model on Verta as a Verta Standard Model either via  convenience functions (for Keras) or by extending [VertaModelBase](https://verta.readthedocs.io/en/master/_autogen/verta.registry.VertaModelBase.html?highlight=VertaModelBase#verta.registry.VertaModelBase).

## 0. Imports

In [1]:
import os
import tensorflow as tf

In [2]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [3]:
import os

# Ensure credentials are set up, if not, use below
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST'] = 
from verta import Client
client = Client(os.environ['VERTA_HOST'])

## 1. Model Training

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [7]:
model.fit(x_train, y_train, epochs=5)

## 2. Register Model

In [8]:
registered_model = client.get_or_create_registered_model(
    name="mnist", labels=["computer-vision", "tensorflow"])

### 2.1 Register from the model object
#### If you are in the same file where you have the model object handy, use the code below to package the model

In [9]:
from verta.environment import Python
model_version_from_obj = registered_model.create_standard_model_from_keras(
    model, environment=Python(requirements=["tensorflow"]), name="v1")

### 2.2 (OR) Register a serialized version of the model using the VertaModelBase

In [10]:
model.save("mnist.tf_saved_model")

In [11]:
from verta.registry import VertaModelBase

class MNISTModel(VertaModelBase):
    def __init__(self, artifacts):
        import tensorflow as tf
        self.model = tf.keras.models.load_model(
            artifacts["mnist_model"])
    
    def predict(self, input_data):
        output = []
        for input_data_point in input_data:
            reshaped_data = tf.reshape(input_data_point, (1, 28, 28))
            output.append(self.model(reshaped_data).numpy().tolist())
        return output

In [12]:
# test locally
mnist_model1 = MNISTModel({"mnist_model" : "mnist.tf_saved_model/"})
mnist_model1.predict([x_test[0]])

In [13]:
model_version_from_cls = registered_model.create_standard_model(
    MNISTModel, 
    environment=Python(["tensorflow"]), 
    name="v2", 
    artifacts={"mnist_model" : "mnist.tf_saved_model/"}
)

### 2.3 (OR) Register a serialized version of the model using the VertaModelBase (Variation: take in a base64 encoded input vs. a tensor)

In [14]:
class MNISTModel2(VertaModelBase):
    def __init__(self, artifacts):
        import tensorflow as tf
        import base64
        self.model = tf.keras.models.load_model(artifacts["mnist_model"])
    
    def predict(self, input_data):
        # decode base64
        import base64
        output = []
        for input_data_point in input_data:
            decoded_data = base64.b64decode(input_data_point["img_bytes"])
            decoded_data = tf.io.decode_image(decoded_data)
            decoded_data = tf.reshape(decoded_data, (1, 28, 28))
            output.append(self.model(decoded_data).numpy().tolist())
        return output

In [15]:
# test locally
import base64
mnist_model2 = MNISTModel2({"mnist_model" : "mnist.tf_saved_model/"})
with open("2.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    print(mnist_model2.predict([{"img_bytes" : encoded_string}]))

In [16]:
model_version_from_cls_base64 = registered_model.create_standard_model(
    MNISTModel2, 
    environment=Python(["tensorflow"]), 
    name="v3", 
    artifacts={"mnist_model" : "mnist.tf_saved_model/"}
)

## 3. Deploy model to endpoint

In [17]:
mnist_endpoint = client.get_or_create_endpoint("mnist")
mnist_endpoint.update(model_version_from_obj, wait=True)
deployed_model = mnist_endpoint.get_deployed_model()
deployed_model.predict([x_test[0]])

In [18]:
mnist_endpoint = client.get_or_create_endpoint("mnist")
mnist_endpoint.update(model_version_from_cls, wait=True)
deployed_model = mnist_endpoint.get_deployed_model()
deployed_model.predict([x_test[0]])

In [19]:
mnist_endpoint = client.get_or_create_endpoint("mnist")
mnist_endpoint.update(model_version_from_cls_base64, wait=True)
deployed_model = mnist_endpoint.get_deployed_model()
with open("2.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    print(deployed_model.predict([{"img_bytes" : encoded_string}]))

---